In [ ]:
# Check and install packages listed
package.check = function(packages){
    lapply(
    packages,
    FUN = function(x) {
      if (!require(x, character.only = TRUE)) {
        install.packages(x, dependencies = TRUE,repos="https://cloud.r-project.org")
        library(x, character.only = TRUE)
      }
    }
  )
}

copytable = function(x,row.names=FALSE,col.names=TRUE,...) {
  write.table(x,"clipboard",sep="/t",row.names=row.names,col.names=col.names,...)
}

# center with mean function for continuous variables
center_w_mean = function(x) {
    xcenter = mean(x)
    x - xcenter
}

cleanData = function(data) {
  data = data[-2,] %>% select(contains(c("Q","condition","randomfield")) & !contains(c("NPS","Browser","Version","System","Resolution","Click")))

  # specifically to remove tutorial stuff
  # data = select(data, !starts_with("QID17"))

  data[1,] = lapply(
      data[1,],
      function(x) {
          gsub(".*statements. - ", "", x)
      }
    )
  data[1,] = lapply(
      data[1,],
      function(x) {
        sub("\r\n.*", "", x)
      }
    )
  data[1,] = lapply(
      data[1,],
      function(x) {
        gsub("Timing - ", "", x)
      }
    )
  data[1,] = lapply(
      data[1,],
      function(x) {
        gsub(" Cognitive Load", "", x)
      }
    )

  # colnames(data)= makeInitials(data[1,])
  colnames(data)= data[1,]
  data = data[-1,]
}

makeInitials = function(charVec) {
  make.unique(vapply(strsplit(toupper(charVec), " "), 
                     function(x) paste(substr(x, 1, 1), collapse = ""), 
                     vector("character", 1L)))
}

In [ ]:
packages = c("tidyverse", "ggpubr", "ggvis", "ggeffects", "lmerTest", "easystats", "pwr")
package.check(packages)

In [ ]:
# Cohen's effect sizes small=.02, medium=.15, large=.35 (Cohen, J. (1988). Statistical power analysis for the behavioral sciences (2nd ed.). Hillsdale, NJ: Lawrence Erlbaum)

# u is the number of coefficients in the model (i.e., number of independent + interaction variables + 1 for cutoff - b0) Therefore, u > 1

# u = 3 because b0 + ConditionC + CovariateC (e.g., ManufactuabilityC)
pwr.f2.test(u = 3, f2 = .35, sig.level = 0.05, power = 0.8)
# sample size = u + roundup(v) + 1

In [ ]:
# disable warnings
options(warn = - 1) 

In [ ]:
# create color palettes

# https://stackoverflow.com/questions/57153428/r-plot-color-combinations-that-are-colorblind-accessible

# opalette = conditionsPalette.colors(palette = "Okabe-Ito")
conditionsPalette = hcl.colors(palette = "plasma",n = 3)

In [ ]:
# custom plot theme
plottheme = theme_pubr()+
            theme(
              legend.position="bottom",
              legend.background = element_rect(fill = "transparent"),
              strip.background = element_blank(),
              # legend.box.background = element_rect(fill = "transparent"),
              panel.background = element_rect(fill = "transparent"),
              panel.grid.major = element_blank(),
              panel.grid.minor = element_blank(),
              plot.background = element_rect(fill = "transparent", color = NA),
              axis.title.y = element_text(margin = margin(r = 20)),
              # text=element_text(size=18, family="serif")
            )         

In [ ]:
data = readxl::read_excel("data.xlsx")
data = cleanData(data)
# fix(data)